In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode

from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder

C:\Users\thami\Documents\GitHub\TCC agrupamento\agrupamento-tcc\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Funções

In [2]:
def remove_acentos(texto):
    return unidecode(str(texto))

In [3]:
def minusculo(texto):
    texto = texto.replace(' ', '_')
    return str(texto).lower()

#### Leitura

In [4]:
df_2022 = pd.read_csv('dados/microdados_2022/dados/MICRODADOS_CADASTRO_CURSOS_2022.CSV',encoding='ISO-8859-1',sep=';')

C:\Users\thami\AppData\Local\Temp\ipykernel_10860\2480471517.py:1: DtypeWarning: Columns (1,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv('dados/microdados_2022/dados/MICRODADOS_CADASTRO_CURSOS_2022.CSV',encoding='ISO-8859-1',sep=';')


In [5]:
df_indicadores = pd.read_csv('dados_processados/indicadores.csv', sep=',')

#### Limpeza e ajuste de dados

In [6]:
df_2022.shape

(573019, 200)

In [7]:
#Decidir se necessita dropar os na ou não
#df_2022 = df_2022.dropna()

In [8]:
df_2022.shape

(573019, 200)

In [9]:
df_2022['TP_MODALIDADE_ENSINO'].unique()

array([2, 1], dtype=int64)

In [10]:
df_2022 = df_2022.query('TP_MODALIDADE_ENSINO == 1').reset_index(drop=True).copy()

In [11]:
df_2022['TP_MODALIDADE_ENSINO'].unique()

array([1], dtype=int64)

In [12]:
cursos_ufrpe = df_2022[df_2022['CO_IES'] == 587][['CO_CURSO', 'NO_CURSO']]

In [13]:
cursos_ufrpe.shape

(46, 2)

In [14]:
cursos_ufrpe.to_csv('dados_processados/cursos_ufrpe.csv', sep=';', index=False)

In [15]:
df_2022

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_DIMENSAO,...,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
0,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,4,0,1,1,1,0,0,0,0,0
1,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,71,0,168,65,66,0,0,0,0,0
4,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36084,2022,Sul,4,Santa Catarina,SC,42,Xaxim,4219705,0,1,...,0,0,46,18,7,7,0,0,0,0
36085,2022,Sul,4,Santa Catarina,SC,42,Xaxim,4219705,0,1,...,0,0,30,3,0,0,0,0,0,0
36086,2022,Sul,4,Santa Catarina,SC,42,Xaxim,4219705,0,1,...,0,0,75,14,18,18,0,0,0,0
36087,2022,Sul,4,Santa Catarina,SC,42,Xaxim,4219705,0,1,...,0,0,10,0,7,7,0,0,0,0


#### Ajustes dataframe microdados

In [16]:
df_2022.replace({'TP_GRAU_ACADEMICO':{1:'bacharelado',2:'licenciatura',3:'tecnologico',4:'bacharelado e licenciatura'}},inplace=True)

In [17]:
df_2022.columns = df_2022.columns.str.lower()

In [18]:
colunas_categoricas = df_2022.select_dtypes(include=['object']).columns
for coluna in colunas_categoricas:
    df_2022[coluna] = df_2022[coluna].apply(remove_acentos)
    df_2022[coluna] = df_2022[coluna].apply(minusculo)

In [19]:
df_2022

,nu_ano_censo,no_regiao,co_regiao,no_uf,sg_uf,co_uf,no_municipio,co_municipio,in_capital,tp_dimensao,...,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_ing_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica
0,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,4,0,1,1,1,0,0,0,0,0
1,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,71,0,168,65,66,0,0,0,0,0
4,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36084,2022,sul,4,santa_catarina,sc,42,xaxim,4219705,0,1,...,0,0,46,18,7,7,0,0,0,0
36085,2022,sul,4,santa_catarina,sc,42,xaxim,4219705,0,1,...,0,0,30,3,0,0,0,0,0,0
36086,2022,sul,4,santa_catarina,sc,42,xaxim,4219705,0,1,...,0,0,75,14,18,18,0,0,0,0
36087,2022,sul,4,santa_catarina,sc,42,xaxim,4219705,0,1,...,0,0,10,0,7,7,0,0,0,0


In [20]:
print(df_indicadores.dtypes)

co_curso         int64
enade_faixa    float64
cpc_faixa      float64
dtype: object


In [21]:
df_indicadores.columns

Index(['co_curso', 'enade_faixa', 'cpc_faixa'], dtype='object')

In [22]:
#Para evitar enade_faixa e cpc_faixa NaN não foi utilizado o left no merge
resultado = pd.merge(df_2022, df_indicadores[['co_curso', 'enade_faixa', 'cpc_faixa']], on='co_curso')

In [23]:
resultado

,nu_ano_censo,no_regiao,co_regiao,no_uf,sg_uf,co_uf,no_municipio,co_municipio,in_capital,tp_dimensao,...,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_ing_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica,enade_faixa,cpc_faixa
0,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,550,37,543,52,4,0,4,4,5.0,5.0
1,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,254,4,250,15,0,0,0,0,4.0,4.0
2,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,253,5,251,22,5,0,5,0,5.0,4.0
3,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,176,9,172,10,0,0,0,0,5.0,4.0
4,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,55,3,55,4,0,0,0,0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,2022,sul,4,santa_catarina,sc,42,sao_francisco_do_sul,4216206,0,1,...,0,0,0,0,0,0,0,0,4.0,3.0
6507,2022,sul,4,santa_catarina,sc,42,sao_jose,4216602,0,1,...,27,3,27,3,1,0,1,0,4.0,4.0
6508,2022,sul,4,santa_catarina,sc,42,tubarao,4218707,0,1,...,1,0,0,0,0,0,0,0,4.0,4.0
6509,2022,sul,4,santa_catarina,sc,42,videira,4219309,0,1,...,16,1,15,1,0,0,0,0,3.0,3.0


In [24]:
resultado.shape

(6511, 202)

In [25]:
resultado.describe()

,nu_ano_censo,co_regiao,co_uf,co_municipio,in_capital,tp_dimensao,tp_organizacao_academica,tp_categoria_administrativa,tp_rede,co_ies,...,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_ing_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica,enade_faixa,cpc_faixa
count,6511.0,6511.000000,6511.000000,6.511000e+03,6511.000000,6511.0,6511.000000,6511.000000,6511.0,6511.000000,...,6511.000000,6511.000000,6511.000000,6511.000000,6511.000000,6511.000000,6511.000000,6511.000000,6511.000000,6511.000000
mean,2022.0,2.947934,32.757795,3.289977e+06,0.267394,1.0,1.590232,1.486408,1.0,2416.967593,...,42.954692,3.281984,41.001382,5.756105,0.333129,0.004761,0.328214,0.022424,3.371986,3.478728
std,0.0,1.176142,10.715760,1.072597e+06,0.442633,0.0,1.149149,0.922042,0.0,4949.374826,...,70.958528,8.660860,67.992799,11.694240,1.673432,0.119428,1.665204,0.186621,1.109036,0.887947
min,2022.0,1.000000,11.000000,1.100023e+06,0.000000,1.0,1.000000,1.000000,1.0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2022.0,2.000000,25.000000,2.507507e+06,0.000000,1.0,1.000000,1.000000,1.0,547.000000,...,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000
50%,2022.0,3.000000,31.000000,3.170206e+06,0.000000,1.0,1.000000,1.000000,1.0,588.000000,...,20.000000,0.000000,20.000000,2.000000,0.000000,0.000000,0.000000,0.000000,3.000000,4.000000
75%,2022.0,4.000000,41.000000,4.115804e+06,1.000000,1.0,1.000000,2.000000,1.0,1807.000000,...,53.000000,3.000000,51.000000,7.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000
max,2022.0,5.000000,53.000000,5.300108e+06,1.000000,1.0,5.000000,7.000000,1.0,25352.000000,...,1215.000000,339.000000,1177.000000,418.000000,40.000000,7.000000,40.000000,4.000000,5.000000,5.000000


In [26]:
resultado['enade_faixa'].unique()

array([5., 4., 3., 2., 0., 1.])

In [27]:
print(resultado.dtypes)

nu_ano_censo               int64
no_regiao                 object
co_regiao                  int64
no_uf                     object
sg_uf                     object
                          ...   
qt_ing_mob_academica       int64
qt_mat_mob_academica       int64
qt_conc_mob_academica      int64
enade_faixa              float64
cpc_faixa                float64
Length: 202, dtype: object


In [28]:
if resultado.isnull().all().any():
    print('Há pelo menos uma coluna com dados ausentes.')
else:
    print('Não há colunas com dados ausentes.')

Não há colunas com dados ausentes.


In [29]:
df = resultado.drop(['no_regiao', 'no_uf', 'sg_uf', 'no_cine_rotulo', 'co_cine_rotulo', 'no_cine_area_especifica', 'no_cine_area_detalhada',
                     'co_cine_area_geral', 'co_cine_area_especifica', 'co_cine_area_detalhada', 'co_municipio'],axis=1)

In [30]:
df.shape

(6511, 191)

#### Tratamento

In [31]:
#Primeiro passo é identificar os outliers e substituí-los

In [32]:
def detect_outliers_iqr(data, threshold=1.5):
    """
    Detecta outliers em uma série de dados usando o método do intervalo interquartil (IQR).
    
    Parâmetros:
        data (pd.Series): A série de dados para verificar outliers.
        threshold (float): O fator multiplicativo do IQR usado para definir o limite para outliers.
        
    Retorna:
        outliers (int): O número de amostras com outliers.
    """
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    return len(outliers)

In [33]:
# Calcula a quantidade de amostras com outliers para cada coluna numérica
outliers_count = {}
for column in df.select_dtypes(include=['number']):
    outliers_count[column] = detect_outliers_iqr(df[column])

# Exibe a quantidade de amostras com outliers por coluna
for column, count in outliers_count.items():
    print(f"Coluna '{column}': {count} amostras com outliers")

Coluna 'nu_ano_censo': 0 amostras com outliers
Coluna 'co_regiao': 0 amostras com outliers
Coluna 'co_uf': 0 amostras com outliers
Coluna 'in_capital': 0 amostras com outliers
Coluna 'tp_dimensao': 0 amostras com outliers
Coluna 'tp_organizacao_academica': 1426 amostras com outliers
Coluna 'tp_categoria_administrativa': 114 amostras com outliers
Coluna 'tp_rede': 0 amostras com outliers
Coluna 'co_ies': 991 amostras com outliers
Coluna 'co_curso': 104 amostras com outliers
Coluna 'in_gratuito': 326 amostras com outliers
Coluna 'tp_modalidade_ensino': 0 amostras com outliers
Coluna 'tp_nivel_academico': 0 amostras com outliers
Coluna 'qt_curso': 0 amostras com outliers
Coluna 'qt_vg_total': 262 amostras com outliers
Coluna 'qt_vg_total_diurno': 113 amostras com outliers
Coluna 'qt_vg_total_noturno': 173 amostras com outliers
Coluna 'qt_vg_total_ead': 0 amostras com outliers
Coluna 'qt_vg_nova': 279 amostras com outliers
Coluna 'qt_vg_proc_seletivo': 0 amostras com outliers
Coluna 'qt_vg

In [34]:
def replace_outliers_iqr(data, threshold=1.5):
    """
    Substitui outliers em uma série de dados pelos limites inferior ou superior do IQR.
    
    Parâmetros:
        data (pd.Series): A série de dados para substituir outliers.
        threshold (float): O fator multiplicativo do IQR usado para definir o limite para outliers.
        
    Retorna:
        data_with_replaced_outliers (pd.Series): A série de dados com outliers substituídos.
    """
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    
    data_with_replaced_outliers = data.copy()
    data_with_replaced_outliers[data_with_replaced_outliers < lower_bound] = lower_bound
    data_with_replaced_outliers[data_with_replaced_outliers > upper_bound] = upper_bound
    
    return data_with_replaced_outliers

In [35]:
# Substitui os outliers por limites do IQR para cada coluna numérica
for column in df.select_dtypes(include=['number']):
    df[column] = replace_outliers_iqr(df[column])

C:\Users\thami\AppData\Local\Temp\ipykernel_10860\950015012.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_with_replaced_outliers[data_with_replaced_outliers > upper_bound] = upper_bound
C:\Users\thami\AppData\Local\Temp\ipykernel_10860\950015012.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '177.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_with_replaced_outliers[data_with_replaced_outliers > upper_bound] = upper_bound
C:\Users\thami\AppData\Local\Temp\ipykernel_10860\950015012.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '137.5' has dtype incompatible with int64, please exp

In [36]:
# Calcula a quantidade de amostras com outliers para cada coluna numérica
outliers_count = {}
for column in df.select_dtypes(include=['number']):
    outliers_count[column] = detect_outliers_iqr(df[column])

# Exibe a quantidade de amostras com outliers por coluna
for column, count in outliers_count.items():
    print(f"Coluna '{column}': {count} amostras com outliers")

Coluna 'nu_ano_censo': 0 amostras com outliers
Coluna 'co_regiao': 0 amostras com outliers
Coluna 'co_uf': 0 amostras com outliers
Coluna 'in_capital': 0 amostras com outliers
Coluna 'tp_dimensao': 0 amostras com outliers
Coluna 'tp_organizacao_academica': 0 amostras com outliers
Coluna 'tp_categoria_administrativa': 0 amostras com outliers
Coluna 'tp_rede': 0 amostras com outliers
Coluna 'co_ies': 0 amostras com outliers
Coluna 'co_curso': 0 amostras com outliers
Coluna 'in_gratuito': 0 amostras com outliers
Coluna 'tp_modalidade_ensino': 0 amostras com outliers
Coluna 'tp_nivel_academico': 0 amostras com outliers
Coluna 'qt_curso': 0 amostras com outliers
Coluna 'qt_vg_total': 0 amostras com outliers
Coluna 'qt_vg_total_diurno': 0 amostras com outliers
Coluna 'qt_vg_total_noturno': 0 amostras com outliers
Coluna 'qt_vg_total_ead': 0 amostras com outliers
Coluna 'qt_vg_nova': 0 amostras com outliers
Coluna 'qt_vg_proc_seletivo': 0 amostras com outliers
Coluna 'qt_vg_remanesc': 0 amost

In [37]:
df

,nu_ano_censo,co_regiao,co_uf,no_municipio,in_capital,tp_dimensao,tp_organizacao_academica,tp_categoria_administrativa,tp_rede,co_ies,...,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_ing_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica,enade_faixa,cpc_faixa
0,2022,5,53,brasilia,1,1,1,1.0,1,2,...,129.5,7.5,124.5,17.5,0,0,0,0,5.0,5.0
1,2022,5,53,brasilia,1,1,1,1.0,1,2,...,129.5,4.0,124.5,15.0,0,0,0,0,4.0,4.0
2,2022,5,53,brasilia,1,1,1,1.0,1,2,...,129.5,5.0,124.5,17.5,0,0,0,0,5.0,4.0
3,2022,5,53,brasilia,1,1,1,1.0,1,2,...,129.5,7.5,124.5,10.0,0,0,0,0,5.0,4.0
4,2022,5,53,brasilia,1,1,1,1.0,1,2,...,55.0,3.0,55.0,4.0,0,0,0,0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,2022,4,42,sao_francisco_do_sul,0,1,1,1.0,1,3697,...,0.0,0.0,0.0,0.0,0,0,0,0,4.0,3.0
6507,2022,4,42,sao_jose,0,1,1,1.0,1,3162,...,27.0,3.0,27.0,3.0,0,0,0,0,4.0,4.0
6508,2022,4,42,tubarao,0,1,1,1.0,1,3162,...,1.0,0.0,0.0,0.0,0,0,0,0,4.0,4.0
6509,2022,4,42,videira,0,1,1,1.0,1,3697,...,16.0,1.0,15.0,1.0,0,0,0,0,3.0,3.0


In [38]:
df.shape

(6511, 191)

In [39]:
df1 = df.drop(['no_curso','no_municipio','no_cine_area_geral','tp_grau_academico','enade_faixa', 'cpc_faixa'],axis=1)

In [40]:
# Remover todas as colunas com variância zero

In [41]:
sel = VarianceThreshold(threshold=(0))
df_selected = sel.fit_transform(df1)
cols_selected = df1.columns[sel.get_support(indices=True)]
df_selected = pd.DataFrame(df_selected, columns=cols_selected)

In [42]:
df_selected

,co_regiao,co_uf,in_capital,tp_categoria_administrativa,co_ies,co_curso,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_nova,...,qt_conc_procescpublica,qt_conc_procescprivada,qt_apoio_social,qt_ing_apoio_social,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular
0,5.0,53.0,1.0,1.0,2.0,145.0,187.0,139.0,132.0,140.0,...,46.0,28.5,116.0,15.0,106.0,9.0,129.5,7.5,124.5,17.5
1,5.0,53.0,1.0,1.0,2.0,135.0,169.0,169.0,0.0,140.0,...,34.0,27.0,116.0,15.0,106.0,10.0,129.5,4.0,124.5,15.0
2,5.0,53.0,1.0,1.0,2.0,132.0,82.0,82.0,0.0,80.0,...,30.0,28.5,111.0,15.0,106.0,8.0,129.5,5.0,124.5,17.5
3,5.0,53.0,1.0,1.0,2.0,1138293.0,61.0,0.0,61.0,60.0,...,21.0,28.5,65.0,12.0,62.0,5.0,129.5,7.5,124.5,10.0
4,5.0,53.0,1.0,1.0,2.0,151.0,78.0,78.0,0.0,25.0,...,8.0,12.0,33.0,5.0,32.0,1.0,55.0,3.0,55.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,4.0,42.0,0.0,1.0,3697.0,1126033.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6507,4.0,42.0,0.0,1.0,3162.0,1102978.0,56.0,0.0,56.0,36.0,...,7.0,2.0,2.0,0.0,2.0,0.0,27.0,3.0,27.0,3.0
6508,4.0,42.0,0.0,1.0,3162.0,1363832.0,54.0,0.0,54.0,40.0,...,32.0,5.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0
6509,4.0,42.0,0.0,1.0,3697.0,1160123.0,70.0,70.0,0.0,40.0,...,2.0,1.0,18.0,10.0,16.0,0.0,16.0,1.0,15.0,1.0


In [43]:
df_selected.shape

(6511, 105)

In [44]:
# Padronizando os dados com StandardScaler

In [45]:
scaler = StandardScaler()
standard_data = scaler.fit_transform(df_selected.drop('co_curso', axis=1))

In [46]:
df_processed = pd.DataFrame(standard_data, columns=df_selected.drop('co_curso', axis=1).columns)

In [47]:
df_processed

,co_regiao,co_uf,in_capital,tp_categoria_administrativa,co_ies,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_nova,qt_vg_remanesc,...,qt_conc_procescpublica,qt_conc_procescprivada,qt_apoio_social,qt_ing_apoio_social,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular
0,1.744877,1.889158,1.655236,-0.685152,-0.917320,2.504188,2.092985,2.568191,2.510784,0.403705,...,2.454108,2.358058,2.306613,2.198462,2.292917,1.808600,2.348591,2.089838,2.357037,2.308014
1,1.744877,1.889158,1.655236,-0.685152,-0.917320,2.106526,2.752614,-0.776911,2.510784,-0.291365,...,1.482187,2.187837,2.306613,2.198462,2.292917,2.093529,2.348591,0.785367,2.357037,1.867447
2,1.744877,1.889158,1.655236,-0.685152,-0.917320,0.184492,0.839689,-0.776911,0.722204,-0.696823,...,1.158214,2.358058,2.170017,2.198462,2.292917,1.523672,2.348591,1.158073,2.357037,2.308014
3,1.744877,1.889158,1.655236,-0.685152,-0.917320,-0.279447,-0.963297,0.768932,0.126010,-0.754746,...,0.429274,2.358058,0.913340,1.613174,0.983063,0.668888,2.348591,2.089838,2.357037,0.986313
4,1.744877,1.889158,1.655236,-0.685152,-0.917320,0.096123,0.751739,-0.776911,-0.917329,2.228266,...,-0.623640,0.485626,0.039130,0.247503,0.089980,-0.470825,0.490282,0.412661,0.550759,-0.071048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,0.894574,0.862553,-0.604144,-0.685152,1.964023,-1.627080,-0.963297,-0.776911,-1.662570,-0.812669,...,-1.190594,-0.876142,-0.862399,-0.727976,-0.862641,-0.755753,-0.881624,-0.705457,-0.878669,-0.775956
6507,0.894574,0.862553,-0.604144,-0.685152,1.546832,-0.389909,-0.963297,0.642223,-0.589422,0.345783,...,-0.704634,-0.649181,-0.807761,-0.727976,-0.803102,-0.755753,-0.208143,0.412661,-0.176950,-0.247275
6508,0.894574,0.862553,-0.604144,-0.685152,1.546832,-0.434094,-0.963297,0.591540,-0.470183,-0.001753,...,1.320201,-0.308739,-0.807761,-0.532881,-0.803102,-0.470825,-0.856680,-0.705457,-0.878669,-0.775956
6509,0.894574,0.862553,-0.604144,-0.685152,1.964023,-0.080616,0.575837,-0.776911,-0.470183,0.925009,...,-1.109601,-0.762662,-0.370656,1.222982,-0.386330,-0.755753,-0.482524,-0.332751,-0.488825,-0.599729


In [48]:
df_processed.shape

(6511, 104)

In [49]:
df_processed['co_curso'] = df['co_curso']
df_processed['no_curso'] = df['no_municipio'] + '_' + df['tp_grau_academico'] + '_' + df['no_curso']
df_processed['no_cine_area_geral'] = df['no_cine_area_geral']
df_processed['tp_grau_academico'] = df['tp_grau_academico']
df_processed['enade_faixa'] = df['enade_faixa']
df_processed['cpc_faixa'] = df['cpc_faixa']

In [50]:
# Calculando a correlação entre colunas e excluindo as que tem correlação maior que o limite determinado (0.8)

In [51]:
limit = 0.8
corr = df_processed.corr(numeric_only=True)
mask = np.triu(
    np.ones(corr.shape), k=1
).astype(bool)
corr_no_dial = corr.where(mask)
coll = [
    c
    for c in corr_no_dial.columns
    if any(abs(corr_no_dial[c])>limit)
]

In [52]:
coll

['co_uf',
 'qt_vg_nova',
 'qt_inscrito_total_diurno',
 'qt_insc_vg_nova',
 'qt_ing_fem',
 'qt_ing_masc',
 'qt_ing_diurno',
 'qt_ing_noturno',
 'qt_ing_vg_nova',
 'qt_ing_18_24',
 'qt_ing_branca',
 'qt_mat',
 'qt_mat_fem',
 'qt_mat_masc',
 'qt_mat_diurno',
 'qt_mat_noturno',
 'qt_mat_18_24',
 'qt_mat_25_29',
 'qt_mat_35_39',
 'qt_mat_40_49',
 'qt_mat_branca',
 'qt_mat_parda',
 'qt_conc_fem',
 'qt_conc_diurno',
 'qt_conc_noturno',
 'qt_conc_18_24',
 'qt_conc_25_29',
 'qt_conc_branca',
 'qt_ing_nacbras',
 'qt_mat_nacbras',
 'qt_conc_nacbras',
 'qt_mat_deficiente',
 'qt_ing_rvredepublica',
 'qt_ing_rvetnico',
 'qt_ing_rvsocial_rf',
 'qt_mat_reserva_vaga',
 'qt_mat_rvredepublica',
 'qt_mat_rvetnico',
 'qt_mat_rvsocial_rf',
 'qt_conc_reserva_vaga',
 'qt_conc_rvredepublica',
 'qt_conc_rvetnico',
 'qt_conc_rvsocial_rf',
 'qt_ing_procescpublica',
 'qt_mat_procescpublica',
 'qt_mat_procescprivada',
 'qt_conc_procescpublica',
 'qt_conc_procescprivada',
 'qt_mat_apoio_social',
 'qt_mat_ativ_extrac

In [53]:
df_processed = df_processed.drop(coll, axis=1)

In [54]:
df_processed['co_curso'] = df['co_curso']

In [55]:
df_processed.shape

(6511, 60)

In [56]:
df_processed

,co_regiao,in_capital,tp_categoria_administrativa,co_ies,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_remanesc,qt_inscrito_total,qt_inscrito_total_noturno,...,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_conc_ativ_extracurricular,co_curso,no_curso,no_cine_area_geral,tp_grau_academico,enade_faixa,cpc_faixa
0,1.744877,1.655236,-0.685152,-0.917320,2.504188,2.092985,2.568191,0.403705,2.202643,2.144538,...,1.808600,2.348591,2.089838,2.308014,145,brasilia_bacharelado_administracao,"negocios,_administracao_e_direito",bacharelado,5.0,5.0
1,1.744877,1.655236,-0.685152,-0.917320,2.106526,2.752614,-0.776911,-0.291365,0.445931,-0.683050,...,2.093529,2.348591,0.785367,1.867447,135,brasilia_bacharelado_agronomia,"agricultura,_silvicultura,_pesca_e_veterinaria",bacharelado,4.0,4.0
2,1.744877,1.655236,-0.685152,-0.917320,0.184492,0.839689,-0.776911,-0.696823,1.522730,-0.683050,...,1.523672,2.348591,1.158073,2.308014,132,brasilia_bacharelado_arquitetura_e_urbanismo,"engenharia,_producao_e_construcao",bacharelado,5.0,4.0
3,1.744877,1.655236,-0.685152,-0.917320,-0.279447,-0.963297,0.768932,-0.754746,0.117361,2.144538,...,0.668888,2.348591,2.089838,0.986313,1138293,brasilia_bacharelado_arquitetura_e_urbanismo,"engenharia,_producao_e_construcao",bacharelado,5.0,4.0
4,1.744877,1.655236,-0.685152,-0.917320,0.096123,0.751739,-0.776911,2.228266,-0.673159,-0.683050,...,-0.470825,0.490282,0.412661,-0.071048,151,brasilia_licenciatura_artes_visuais,educacao,licenciatura,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,0.894574,-0.604144,-0.685152,1.964023,-1.627080,-0.963297,-0.776911,-0.812669,-0.965945,-0.683050,...,-0.755753,-0.881624,-0.705457,-0.775956,1126033,sao_francisco_do_sul_tecnologico_redes_de_comp...,computacao_e_tecnologias_da_informacao_e_comun...,tecnologico,4.0,3.0
6507,0.894574,-0.604144,-0.685152,1.546832,-0.389909,-0.963297,0.642223,0.345783,-0.497488,0.745626,...,-0.755753,-0.208143,0.412661,-0.247275,1102978,sao_jose_licenciatura_quimica,educacao,licenciatura,4.0,4.0
6508,0.894574,-0.604144,-0.685152,1.546832,-0.434094,-0.963297,0.591540,-0.001753,-0.198197,1.658391,...,-0.470825,-0.856680,-0.705457,-0.775956,1363832,tubarao_tecnologico_analise_e_desenvolvimento_...,computacao_e_tecnologias_da_informacao_e_comun...,tecnologico,4.0,4.0
6509,0.894574,-0.604144,-0.685152,1.964023,-0.080616,0.575837,-0.776911,0.925009,-0.429172,-0.683050,...,-0.755753,-0.482524,-0.332751,-0.599729,1160123,videira_bacharelado_ciencia_da_computacao,computacao_e_tecnologias_da_informacao_e_comun...,bacharelado,3.0,3.0


In [57]:
# Codificando as colunas categóricas usando o OneHotEncoder

In [58]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [59]:
categoricas = ['no_cine_area_geral','tp_grau_academico', 'enade_faixa', 'cpc_faixa']

In [60]:
matriz_enc = enc.fit_transform(df_processed[categoricas])

In [61]:
df_enc = pd.DataFrame(matriz_enc, columns=enc.get_feature_names_out())

In [62]:
X = pd.concat([df_enc.reset_index(drop=True),df_processed.drop(columns=categoricas).reset_index(drop=True)],axis=1)

In [63]:
X

,"no_cine_area_geral_agricultura,_silvicultura,_pesca_e_veterinaria",no_cine_area_geral_artes_e_humanidades,"no_cine_area_geral_ciencias_naturais,_matematica_e_estatistica","no_cine_area_geral_ciencias_sociais,_comunicacao_e_informacao",no_cine_area_geral_computacao_e_tecnologias_da_informacao_e_comunicacao_(tic),no_cine_area_geral_educacao,"no_cine_area_geral_engenharia,_producao_e_construcao","no_cine_area_geral_negocios,_administracao_e_direito",no_cine_area_geral_saude_e_bem-estar,no_cine_area_geral_servicos,...,qt_sit_transferido,qt_ing_procescprivada,qt_apoio_social,qt_ing_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_conc_ativ_extracurricular,co_curso,no_curso
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.721586,1.425169,2.306613,2.198462,1.808600,2.348591,2.089838,2.308014,145,brasilia_bacharelado_administracao
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.721586,-0.265453,2.306613,2.198462,2.093529,2.348591,0.785367,1.867447,135,brasilia_bacharelado_agronomia
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.580812,-0.632980,2.170017,2.198462,1.523672,2.348591,1.158073,2.308014,132,brasilia_bacharelado_arquitetura_e_urbanismo
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.559962,-0.779990,0.913340,1.613174,0.668888,2.348591,2.089838,0.986313,1138293,brasilia_bacharelado_arquitetura_e_urbanismo
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.559962,-0.779990,0.039130,0.247503,-0.470825,0.490282,0.412661,-0.071048,151,brasilia_licenciatura_artes_visuais
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.559962,-0.927001,-0.862399,-0.727976,-0.755753,-0.881624,-0.705457,-0.775956,1126033,sao_francisco_do_sul_tecnologico_redes_de_comp...
6507,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.559962,-0.338959,-0.807761,-0.727976,-0.755753,-0.208143,0.412661,-0.247275,1102978,sao_jose_licenciatura_quimica
6508,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.559962,-0.044937,-0.807761,-0.532881,-0.470825,-0.856680,-0.705457,-0.775956,1363832,tubarao_tecnologico_analise_e_desenvolvimento_...
6509,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.559962,-0.118443,-0.370656,1.222982,-0.755753,-0.482524,-0.332751,-0.599729,1160123,videira_bacharelado_ciencia_da_computacao


In [64]:
X.shape

(6511, 79)

In [65]:
X = X.drop('in_capital', axis=1)

In [66]:
X.shape

(6511, 78)

In [67]:
#X.columns

In [68]:
###'qt_vg_total_diurno','qt_vg_total_noturno','qt_inscrito_total_diurno', 'qt_ing', 'qt_ing_25_29', 'qt_ing_30_34', 'qt_ing_40_49', 'qt_ing_50_59', 'qt_mat_60_mais', 'qt_mat_cornd', 'qt_conc_diurno', 'qt_conc_30_34', 'qt_conc_preta', 'qt_mat_amarela', 'qt_mat_indigena'

In [69]:
# Dropando colunas que não serão utilizadas
X = X.drop(['qt_vg_total_diurno',
 'qt_vg_total_noturno',
 'qt_ing_25_29',
 'qt_ing_30_34',
 'qt_ing_40_49',
 'qt_ing_50_59',
 'qt_mat_60_mais',
 'qt_mat_cornd',
 'qt_conc_30_34',
 'qt_conc_preta',
 'qt_mat_amarela',
 'qt_mat_indigena',
 'co_regiao',
 'tp_categoria_administrativa',
 'co_ies',
 'qt_inscrito_total_noturno',
 'qt_ing_vestibular',
 'qt_ing_enem',
 'qt_ing_vg_remanesc',
 'qt_ing_35_39',
 'qt_ing_parda',
 'qt_mat_50_59',
 'qt_conc_35_39',
 'qt_conc_40_49',
 'qt_conc_cornd',
 'qt_aluno_deficiente',
 'qt_mat_rvpdef',
 'qt_ing_ativ_extracurricular',
 'qt_mat_nacestrang',
 'qt_ing',
 'qt_mat_30_34',
 'qt_mat_preta',
 'qt_conc_masc',
 'qt_conc_parda',
 'qt_sit_transferido',
 'qt_ing_procescprivada',
 'qt_ing_apoio_social',
 'qt_conc_ativ_extracurricular'],axis=1)

In [70]:
X.columns

Index(['no_cine_area_geral_agricultura,_silvicultura,_pesca_e_veterinaria',
       'no_cine_area_geral_artes_e_humanidades',
       'no_cine_area_geral_ciencias_naturais,_matematica_e_estatistica',
       'no_cine_area_geral_ciencias_sociais,_comunicacao_e_informacao',
       'no_cine_area_geral_computacao_e_tecnologias_da_informacao_e_comunicacao_(tic)',
       'no_cine_area_geral_educacao',
       'no_cine_area_geral_engenharia,_producao_e_construcao',
       'no_cine_area_geral_negocios,_administracao_e_direito',
       'no_cine_area_geral_saude_e_bem-estar', 'no_cine_area_geral_servicos',
       'tp_grau_academico_bacharelado', 'tp_grau_academico_licenciatura',
       'tp_grau_academico_tecnologico', 'enade_faixa_1.5', 'enade_faixa_2.0',
       'enade_faixa_3.0', 'enade_faixa_4.0', 'enade_faixa_5.0',
       'cpc_faixa_1.5', 'cpc_faixa_2.0', 'cpc_faixa_3.0', 'cpc_faixa_4.0',
       'cpc_faixa_5.0', 'qt_vg_total', 'qt_vg_remanesc', 'qt_inscrito_total',
       'qt_insc_vg_remanesc', '

In [71]:
X.shape

(6511, 40)

In [72]:
#Salvando os dados
X.to_csv('dados_processados/cursos-pos-processamento.csv',sep=';',index=False)